In [1]:
import sys
import os

# Add the path to the aqi package
aqi_path = os.path.abspath('../station')
sys.path.append(aqi_path)

from fetch import fetch_data
from etl import validate_data, parse_data, clean_data, push_to_db

In [2]:
aqi_data = fetch_data()
print(aqi_data['status'])
print(len(aqi_data['data']))

<Response [200]>
{'status': 'ok', 'data': [{'lat': 19.858547, 'lon': 99.90148, 'uid': 9587, 'aqi': '67', 'station': {'name': 'Chiangrai - Gaia Station 07, Thailand', 'time': '2025-02-23T21:00:00+09:00'}}, {'lat': 5.629631, 'lon': 100.467771, 'uid': 2582, 'aqi': '35', 'station': {'name': 'Bakar Arang, Sg. Petani, Kedah, Malaysia', 'time': '2025-02-23T21:00:00+09:00'}}, {'lat': 20.320741, 'lon': 105.745489, 'uid': 13763, 'aqi': '26', 'station': {'name': 'Ninh Bình/Cúc Phương, Đông Phong, Nho Quan - KTTV - Trạm cảm biến, Vietnam', 'time': '2025-02-23T21:00:00+09:00'}}, {'lat': 14.022376, 'lon': 99.536195, 'uid': 10481, 'aqi': '135', 'station': {'name': 'Pak Prak, Kanchanaburi, Kanchanaburi, Thailand (ต.ปากแพรก อ.เมือง จ.กาญจนบุรี)', 'time': '2025-02-23T20:00:00+09:00'}}, {'lat': 14.476214, 'lon': 100.086095, 'uid': 13633, 'aqi': '45', 'station': {'name': 'Suphan Buri Provincial Central Stadium, Thailand (สนามกีฬากลางจังหวัดสุพรรณบุรี)', 'time': '2025-02-23T20:00:00+09:00'}}, {'lat': 13.57

In [6]:
validate_data(aqi_data)

In [7]:
parse_aqi_data = parse_data(aqi_data)
print(parse_aqi_data[:3])

     uid aqi   latitude   longitude  \
0  10482  34   6.624747  100.066159   
1   8322  88  13.054551  101.098128   
2  13629  28   8.426923   99.961469   

                                        station_name                 timestamp  
0  Phiman, Satun, Satun, Thailand (ต.พิมาน อ.เมือ... 2025-02-23 19:00:00+09:00  
1  Health Promotion Hospital Ban Khao Hin,Chonbur... 2025-02-23 19:00:00+09:00  
2  Elderly Health Rehabilitation Center, Thailand... 2025-02-23 19:00:00+09:00  


In [8]:
clean_aqi_data = clean_data(parse_aqi_data)
print(clean_aqi_data[:3])

     uid   aqi   latitude   longitude  \
0  10482  34.0   6.624747  100.066159   
1   8322  88.0  13.054551  101.098128   
2  13629  28.0   8.426923   99.961469   

                                        station_name                 timestamp  
0  Phiman, Satun, Satun, Thailand (ต.พิมาน อ.เมือ... 2025-02-23 19:00:00+09:00  
1  Health Promotion Hospital Ban Khao Hin,Chonbur... 2025-02-23 19:00:00+09:00  
2  Elderly Health Rehabilitation Center, Thailand... 2025-02-23 19:00:00+09:00  


In [10]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Push the cleaned AQI data to the database
push_data = push_to_db(conn, clean_aqi_data)

# Close the connection
conn.close()